In [2]:
import numpy as np
import pandas as pd

from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import GridSearchCV, cross_val_predict
from sklearn.metrics import classification_report

from sklearn.model_selection import train_test_split

import os
import pickle

In [3]:
import re
import spacy

In [8]:
import json_lines
import json

### Stopwords

In [3]:
ordinals = ['1st', '2nd', '3rd', '4th']
for i in range(5,31):
    ordinals.append(str(i) + 'th')
    ordinals.append('0' + str(i) + 'th') 
    

In [4]:
number_stopwords = [str(i) for i in range(10001)] + ['0'+str(i) for i in range(100)] + ['000'] + ordinals

In [104]:
df = pd.read_excel('Data/20181001-newindianexpress_sentence_classification_adjudicated_20181218.xlsx')

In [105]:
df = df[np.logical_not(np.isnan(np.array(df['label'])))]

In [106]:
len(df)

8337

In [103]:
for i, row in df.iterrows():
    row['sentence'] = re.sub("http\S*\s", "", row['sentence'])
    row['sentence'] = re.sub("((url)*\s*:\s*)*http\S*", "", row['sentence'])
    
    if row['sentence'] == "":
        df=df.drop([i])
        

In [49]:
len(df)

7796

In [9]:
df1_sents= df['sentence']
df1_y= df['label']

In [10]:
len(df1_sents)

7796

import pickle

data_path = 'Data/first_data.pickle'



prepared_first = {}

prepared_first['X'] = np.array(df1_sents)
prepared_first['y'] = np.array(df1_y)
prepared_first['stopwords'] = np.array(number_stopwords)

with open(data_path, 'wb') as file_:
    pickle.dump(prepared_first, file_, protocol=pickle.HIGHEST_PROTOCOL)

In [11]:
## Partition the data

X_, X_test, y_, y_test = train_test_split(np.array(df1_sents), np.array(df1_y), test_size=0.20, random_state=42, stratify=np.array(df1_y))

len(X_), len(X_test)

 

(6236, 1560)

In [12]:
X_tra, X_val, y_tra, y_val = train_test_split(X_, y_, test_size=0.20, random_state=42, stratify=y_)

type(X_tra), len(X_val)

data_path = 'Data/first_data_prepared.pickle'



In [13]:
Data_ = {}

Data_['X_tra'] = X_tra
Data_['y_tra'] = y_tra
Data_['X_val'] = X_val
Data_['y_val'] = y_val
Data_['X_test'] = X_test
Data_['y_test'] = y_test
Data_['stopwords'] = np.array(number_stopwords)

with open(data_path, 'wb') as file_:
    pickle.dump(Data_, file_, protocol=pickle.HIGHEST_PROTOCOL)

## Second Haf of the data

In [14]:
df2 = pd.read_excel('Data/20181126_NewIndianExpress_3Classifier0_Baglan-Ezgi-Balacan-Eylem-Merged_AgreedByAll.xlsx')
df2 = df2[np.logical_not(np.isnan(np.array(df2['label'])))]
df2 = df2[(np.array(df2['label'] == 0))]

In [15]:
nlp = spacy.load('en_core_web_lg')

In [16]:
df2_sents = []
df2_labels = []
for i, row in df2.iterrows():
    row['text'] = row['text'].replace('\n', ' ')
    row['text'] = row['text'].replace('&amp;', 'and')
    row['text'] = row['text'].replace('\xa0', ' ')
    row['text'] = row['text'].replace('-', '')
    row['text'] = row['text'].replace("\'", "")
    row['text'] = row['text'].replace('\"', '')
    row['text'] = row['text'].replace('\“', '')
    row['text'] = re.sub("((url)*\s*:\s*)*http\S*", "", row['text'])
    row['text'] = row['text'].replace('  ', ' ')
    doc = nlp(row['text'])
    for sent in doc.sents:
        df2_sents.append(sent.text) 
        df2_labels.append(0)


In [17]:
len(df2_sents)

8743

In [18]:
df2_sents[:100]

['Britains little prince gets a name 25th July 2013 08:51 AM',
 'The little prince was in need of a name, and now, by George, hes got one.',
 'The announcement Wednesday that Prince William and his wife, Kate, had selected a moniker steeped in British history came as royal officials said the new parents were seeking quiet family time away from the flashbulbs and frenzy that accompanied the birth of their first child.',
 'While the news put to rest intense speculation over what name the couple would choose, the extreme interest around it illustrated how the 2dayold future heir is already on his way to a lifetime of fanfare and public glare.',
 'Kensington Palace said William and Kate were delighted to announce their sons name, adding that the baby will be known as His Royal Highness Prince George of Cambridge.',
 'The name George — borne by six kings — befits the boy now third in line to the British throne and was a favorite among British bookmakers, evoking the steadfastness of the que

In [19]:
sents = df1_sents.tolist() + df2_sents
y = df1_y.tolist() + df2_labels

In [18]:
len(sents)

16539

In [19]:
import pickle

In [20]:
data_path = 'Data/all_data2.pickle'

In [21]:
Data_ = {}

In [22]:
Data_['X'] = np.array(sents)
Data_['y'] = np.array(y)
Data_['stopwords'] = np.array(number_stopwords)

In [23]:
with open(data_path, 'wb') as file_:
    pickle.dump(Data_, file_, protocol=pickle.HIGHEST_PROTOCOL)

## Read pickle

In [24]:
data_path = 'Data/all_data2.pickle'

In [25]:
with open(data_path, 'rb') as file_:
    data_ = pickle.load(file_)

In [26]:
X = data_['X']
y = data_['y']
stopwords = data_['stopwords']

In [27]:
type(y)

numpy.ndarray

## Partition the data

In [28]:
 X_, X_test, y_, y_test = train_test_split(X, y, test_size=0.20, random_state=42, stratify=y)

In [29]:
len(X_), len(X_test)

(13231, 3308)

In [30]:
 X_tra, X_val, y_tra, y_val = train_test_split(X_, y_, test_size=0.20, random_state=42, stratify=y_)

In [31]:
type(X_tra), len(X_val)

(numpy.ndarray, 2647)

In [32]:
data_path = 'Data/prepared_data.pickle'

In [33]:
Data_ = {}

In [34]:
Data_['X_tra'] = X_tra
Data_['y_tra'] = y_tra
Data_['X_val'] = X_val
Data_['y_val'] = y_val
Data_['X_test'] = X_test
Data_['y_test'] = y_test
Data_['stopwords'] = np.array(number_stopwords)

In [35]:
with open(data_path, 'wb') as file_:
    pickle.dump(Data_, file_, protocol=pickle.HIGHEST_PROTOCOL)

# New way

df = pd.read_excel('Data/20181001-newindianexpress_sentence_classification_adjudicated_20181218.xlsx')

df = df[np.logical_not(np.isnan(np.array(df['label'])))]

len(df)

for i, row in df.iterrows():
    row['sentence'] = re.sub("http\S*\s", "", row['sentence'])
    row['sentence'] = re.sub("((url)*\s*:\s*)*http\S*", "", row['sentence'])
    df['sentence'][i] = row['sentence']
    if row['sentence'] == "":
        df=df.drop([i])
df.reset_index(drop=True)
df.to_csv(r'Data/preprocessed.csv')

df = df.drop(['Unnamed: 0', 'Unnamed: 0.1.1'], axis=1)
df.to_csv(r'Data/preprocessed.csv')

In [4]:
df = pd.read_csv("Data/preprocessed.csv") 

In [10]:
df.head()

,Unnamed: 0,url,sent_num,sentence,text,label
0,0,http://www.newindianexpress.com/states/andhra-...,1-1,Anantapur Lawyers to Revive Samaikya Agitation...,Anantapur Lawyers to Revive Samaikya Agitation...,2.0
1,1,http://www.newindianexpress.com/states/andhra-...,1-2,A meeting of lawyers was held in the town on S...,Anantapur Lawyers to Revive Samaikya Agitation...,0.0
2,2,http://www.newindianexpress.com/states/andhra-...,1-3,"Speaking to reporters later, advocates’ leader...",Anantapur Lawyers to Revive Samaikya Agitation...,0.0
3,3,http://www.newindianexpress.com/states/andhra-...,1-4,“Bifurcation is being done without taking into...,Anantapur Lawyers to Revive Samaikya Agitation...,0.0
4,4,http://www.newindianexpress.com/states/andhra-...,1-5,The Bill did not contain any proposals for dev...,Anantapur Lawyers to Revive Samaikya Agitation...,0.0


In [21]:
doc_ids = set()
doc_sents_last = {}
for i, row in df.iterrows():
    doc_num, sent_num = row['sent_num'].split('-')
    doc_ids.add(doc_num)
    doc_sents_last[doc_num] = sent_num

In [60]:
jslines=[]
last_=False
for i, row in df.iterrows():
    doc_num, sent_num = row['sent_num'].split('-')
    row['sentence'] = re.sub("http\S*\s", "", row['sentence'])
    sent = re.sub("((url)*\s*:\s*)*http\S*", "", row['sentence'])
    last_= False
    if not sent == "":
        if sent_num == doc_sents_last[doc_num]:
            last_=True
#             print(doc_num, sent_num)
        jslines.append({'id': str(doc_num)+'000'+str(sent_num),
                        'sentence': sent,
                        'label': int(row['label']),
                        'last':last_})


In [61]:
with open('Data/processedalldata.jsonl', 'w') as outfile:
    for entry in jslines:
        json.dump(entry, outfile)
        outfile.write('\n')

In [11]:
v_data = []
with json_lines.open('Data/processedalldata.jsonl') as reader:
    for obj in reader:
        v_data.append(obj)

In [12]:
len(v_data)

7796

In [55]:
thesis_training_data = v_data[:5981]
thesis_validation_data = v_data[5982:6654]
thesis_test_data = v_data[6655:]

In [56]:
with open('Data/thesis_training_data.jsonl', 'w') as outfile:
    for entry in thesis_training_data:
        json.dump(entry, outfile)
        outfile.write('\n')
        
with open('Data/thesis_validation_data.jsonl', 'w') as outfile:
    for entry in thesis_validation_data:
        json.dump(entry, outfile)
        outfile.write('\n')

with open('Data/thesis_test_data.jsonl', 'w') as outfile:
    for entry in thesis_test_data:
        json.dump(entry, outfile)
        outfile.write('\n')

In [53]:
v_data[7053:7160]

[{'id': '9480001',
  'sentence': 'HC Warns Against Using Temple Issue for Political Mileage 26th February 2014 03:10 PM The Madras High Court today warned that it will take a serious view if the issue concerning a local temple was used for deriving political mileage.',
  'label': 0,
  'last': False},
 {'id': '9480002',
  'sentence': "The First Bench, comprising Acting Chief Justice Sathish K Agnihotri and Justice K Ravichandrabaabu, was hearing a contempt petition filed by social activist 'Traffic' Ramaswamy against authorities for not removing a temple at the gate of the court in spite of the High Court order.",
  'label': 0,
  'last': False},
 {'id': '9480003',
  'sentence': 'Some political organisations had reportedly distributed some posters and pamphlets describing Ramaswamy as anti-Hindu which were displayed by him in the open court.',
  'label': 1,
  'last': False},
 {'id': '9480004',
  'sentence': 'Taking serious note of the matter, the Acting Chief Justice orally warned that i

In [52]:
text = 'HC Warns'
for i,v in enumerate(v_data):
    if text in v['sentence']:
        print(i, v)

7053 {'id': '9480001', 'sentence': 'HC Warns Against Using Temple Issue for Political Mileage 26th February 2014 03:10 PM The Madras High Court today warned that it will take a serious view if the issue concerning a local temple was used for deriving political mileage.', 'label': 0, 'last': False}


In [ ]:
5982 valis data ilk
{'id': '8570001', 
 'sentence': 'Crowd tries to attack US consulate in Pakistan 11th January 
 2009 11:56 PM Thousands of supporters of Pakistanis participate in a rally to condemn
 Israeli military attacks in Karachi.', 'label': 1, 'last': False}
 
 
6654valid data last
{'id': '91600011',
 'sentence': 'The war of words also came on a day when BJP questioned the "silence" of 
 Congress President Sonia Gandhi and Rahul Gandhi on the issue of Robert Vadra\'s land deals 
 and said it is only adding to the suspicion that the "family" has something to avoid answering.',
 'label': 0,
 'last': True}
 
 
 test data ilk 6665
 
 test data son 7795

c2 = 0
for i, stest in enumerate(shared_test_data):
    if stest['last']:
        for j, test in enumerate(test_data):
            if i>1131 or j>1131:
                    pass
            else:
                if test['last']:
                    if test_data[j]['sentence'][:20] in shared_test_data[i]['sentence']:
                        print(test_data[j]['sentence'])
                        print('\n')
                        c2 = c2 +1
                        pass

In [41]:
input_data_dir = '/home/aorus/workspaces/simge/sharedTask/output/Sentence/'
train_path = os.path.join(input_data_dir, 'train_filled.json')
dev_path = os.path.join(input_data_dir, 'dev_filled.json')
test_path = os.path.join(input_data_dir, 'test_filled.json')

shared_test_data = []
with json_lines.open(test_path) as reader:
    for obj in reader:
        shared_test_data.append(obj)


In [42]:
text = 'Taliban claims'
for i,v in enumerate(shared_test_data):
    if text in v['sentence']:
        print(i, v)

In [ ]:

# shared_X_test = []
# for te_data in shared_test_data:
#     shared_X_test.append(te_data['sentence'].replace('\n', '').strip())
